In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Data Loading & Preparation

In [ ]:
dt = pd.read_csv("/kaggle/input/amazon-product-dataset-2020/home/sdf/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv")
print("Data Dimensions are: ", dt.shape)
print("Columns: ", dt.columns)

In [ ]:
dt.dropna(subset=['Category'], axis=0, inplace=True)
print(dt.info())

Data contains 1048574 rows but maximum columns contain 584524 records. 

Half of row are completely empty, so we will drop them. The tricky part is we can't drop all na rows as actual data set  also contain few NA entries. We need to keep them.
We will drop NA values where all entries are Null. 

Also, we will drop last 5 empty columns.

In [ ]:
dt.head()

In [ ]:
dt['Category'] = dt['Category'].str.split("|").str[0].str.strip()

In [ ]:
dt.head(30)

In [ ]:
data = dt[['Product Name', 'Category']]

In [ ]:
data['Category'].unique()

In [ ]:
data = data.drop_duplicates()
data.head()

In [ ]:
#filter_products

In [ ]:
filter_products = """Category == 'Electronics' or Category == 'Cell Phones & Accessories' or Category == 'Beauty & Personal Care' or Category == 'Baby Products' or Category == 'Pet Supplies' or Category == 'Grocery & Gourmet Food' or Category == 'Clothing, Shoes & Jewelry'"""

data = data.query(filter_products)

In [ ]:
data['Category'] = data['Category'].replace("Cell Phones & Accessories", "Electronics")
data['Category'] = data['Category'].replace("Clothing, Shoes & Jewelry", "Fashion")
data['Category'] = data['Category'].replace("Beauty & Personal Care", "Personal Care & Grooming")
data['Category'] = data['Category'].replace("Baby Products", "Personal Care & Grooming")
data['Category'] = data['Category'].replace("Pet Supplies", "Grocery")
data['Category'] = data['Category'].replace("Grocery & Gourmet Food", "Grocery")

In [ ]:
data.Category.unique()

In [ ]:
data.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
#nltk.download(‘stopwords’)


In [ ]:
data['Product Name'] = data['Product Name'].astype(str)

In [ ]:
data['Product Name'] = data['Product Name'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
## remove punctuation
data['Product Name'] = data['Product Name'].str.replace('[^\w\s]','')

In [ ]:
stop = stopwords.words('english')
data['Product Name'] = data['Product Name'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
st = PorterStemmer()
data['Product Name'] = data['Product Name'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(data['Product Name']).toarray()
y = data.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred_NB = classifier.predict(X_test)
y_pred_NB

In [ ]:
cm_NB = confusion_matrix(y_test, y_pred_NB) 
cm_NB

In [ ]:
cf_NB = classification_report(y_test, y_pred_NB)
cf_NB

In [ ]:
test_data = pd.read_excel('../input/search-terms/Search terms.xlsx')

In [ ]:
test_data.head()

In [ ]:
test_data['Search Term'] = test_data['Search Term'].astype(str)

In [ ]:
## remove punctuation
test_data['Search Term'] = test_data['Search Term'].str.replace('[^\w\s]','')

In [ ]:
stop = stopwords.words('english')
test_data['Search Term'] = test_data['Search Term'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
st = PorterStemmer()
test_data['Search Term'] = test_data['Search Term'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [ ]:
test_dataset = cv.fit_transform(test_data['Search Term']).toarray()

In [ ]:
test_data['Category'] = classifier.predict(test_dataset) 

In [ ]:
test_data.head(20)

In [ ]:
test_data.to_excel('question_2.xlsx')